In [11]:
import sys
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen

In [7]:
url_base = 'https://www.beermile.com/display/'
max_event_id = 151381

In [16]:
%%time

# Find which events are actually races
is_event = []
not_event = []

# Look for all events by numerical ID
for i in range(1, max_event_id):
    # Website uses sequential 
    event = f'event_{i}'
    url = f'{url_base}{event}'
    result = requests.get(url)
    page = urlopen(result.url).read()
    a = bs(page, 'html.parser')
    
    
    if (a.title.string == 'Race Not Available'):
        not_event.append(i)
#         print(f'\rNot event{i}')
    else:
        is_event.append(i)
        
    print('\rIs  event: {}'.format(i))
    sys.stdout.flush()
    
    if i> 10:
        break


Is  event: 1
Is  event: 2
Is  event: 3
Is  event: 4
Is  event: 5
Is  event: 6
Is  event: 7
Is  event: 8
Is  event: 9
Is  event: 10
Is  event: 11
CPU times: user 678 ms, sys: 26 ms, total: 704 ms
Wall time: 18.5 s


In [9]:
# Write the results to a file
with open('events.txt', 'w') as f:
    for item in is_event:
        f.write(f'{item}\n')

with open('not_events.txt', 'w') as g:
    for item in not_event:
        g.write(f'{item}\n')